In [1]:
mitos = ["Menggunakan kacamata atau lensa kontak dapat membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.",
"Duduk terlalu dekat dengan layar komputer tidak merusak mata secara permanen.",
"Menggunakan wortel secara berlebihan akan meningkatkan penglihatan dengan cepat.",
"Bersepeda dapat menyebabkan kebutaan.",
"Mata hanya perlu diperiksa jika ada masalah penglihatan.",
"Menggunakan tetes mata secara berlebihan dapat menyebabkan ketergantungan.",
"Membaca dalam cahaya redup tidak merusak mata secara permanen.",
"Menggosok mata saat gatal dapat menyebabkan penyebaran infeksi.",
"Istirahat sebentar setelah melihat layar tidak akan memperbaiki masalah penglihatan.",
"Terlalu banyak membaca dalam cahaya rendah tidak akan merusak mata.",
]

fakta = ["Paparan sinar ultraviolet dari sinar matahari dapat menyebabkan kerusakan mata.",
"Diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan.",
"Perubahan penglihatan terjadi secara alami seiring bertambahnya usia.",
"Merokok dapat meningkatkan risiko terjadinya katarak dan degenerasi makula.",
"Konsumsi makanan bergizi, seperti makanan kaya vitamin A, C, E, dan mineral seperti seng, dapat membantu menjaga kesehatan mata.",
"Istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering.",
"Penggunaan komputer dan perangkat elektronik lainnya dalam jangka waktu yang lama dapat menyebabkan Computer Vision Syndrome (CVS), menyebabkan gejala seperti mata kering, sakit kepala, dan ketegangan leher.",
"Bentuk fisik mata setiap orang berbeda.",
"Kacamata hitam yang baik melindungi mata dari sinar UV dan meminimalkan paparan sinar biru dari layar komputer atau perangkat elektronik lainnya.",
"Deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius."
]

In [10]:
import gensim
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Conv1D, MaxPooling1D
from keras.models import load_model


In [3]:
# Definisi Function
def clean_and_lower(text):
    cleaned_text = ''.join(char.lower() for char in text if char.isalnum() or char.isspace())
    return cleaned_text

# Word2Vec Model
def word2vec_model(sentences):
    words = [sentence.split() for sentence in sentences]
    model = gensim.models.Word2Vec(words, vector_size=100, window=5, min_count=1, workers=4)
    return model

# Convert sentences to average word vectors
def get_avg_word_vectors(sentences, model):
    avg_vectors = []
    for sentence in sentences:
        vectors = []
        for word in sentence.split():
            if word in model.wv:
                vectors.append(model.wv[word])
        if vectors:
            avg_vectors.append(np.mean(vectors, axis=0))
        else:
            avg_vectors.append(np.zeros(model.vector_size))
    return np.array(avg_vectors)

# Build and train a simple neural network
def build_model(input_dim):
    model = Sequential()
    model.add(Conv1D(2, 2, 1, activation='relu', input_dim=input_dim))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(16, 16, 1, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(32, 32, 1, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
    return model

In [4]:
# Kasih target setiap kalimat, 0 = mitos, 1 = fakta
labelledMitos = [(kalimat, 0) for kalimat in mitos]
labelledFakta = [(kalimat, 1) for kalimat in fakta]
print(labelledMitos[0])
print(labelledFakta[0])

('Menggunakan kacamata atau lensa kontak terlalu sering dapat membuat mata malas. Faktanya, kacamata atau lensa kontak membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.', 0)
('Paparan sinar ultraviolet (UV) dari sinar matahari dapat menyebabkan kerusakan mata. Penggunaan kacamata hitam yang memiliki perlindungan UV dapat membantu mengurangi risiko ini.', 1)


In [5]:
# gabung fakta dan mitos, lalu shuffle
trainData = labelledMitos + labelledFakta
random.shuffle(trainData)
print(trainData[0])

('Diabetes dapat mempengaruhi kesehatan mata. Kondisi ini dapat menyebabkan retinopati diabetik, yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan.', 1)


In [6]:
# preprocess, hilangkan yang bukan alpha numerik dan lower
cleanData = [(clean_and_lower(kalimat), label) for kalimat, label in trainData]
random.shuffle(cleanData)
print(cleanData[0])

('konsumsi makanan bergizi seperti makanan kaya vitamin a c e dan mineral seperti seng dapat membantu menjaga kesehatan mata', 1)


In [7]:
sentences = [data for data, label in cleanData]
labels = [label for data, label in cleanData]
print(len(sentences))
print(labels[0])

20
1


In [8]:
# Train & evaluate model
word2vec = word2vec_model(sentences) # ajarin model sentence word2vec
X_train_avg = get_avg_word_vectors(sentences, word2vec)
model = build_model(input_dim=X_train_avg.shape[1])
model.fit(X_train_avg, np.array(labels), epochs=10, verbose=1)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6931 - accuracy: 0.4500
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6928 - accuracy: 0.9000
Epoch 3/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6925 - accuracy: 0.9500
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6924 - accuracy: 0.9500
Epoch 5/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6922 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6921 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6920 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6918 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6917 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6915 - accuracy: 1.0000


In [9]:
# Save model
model.save("w2v.h5")
word2vec.save("w2v.bin")

In [16]:
# Coba hasil
loaded_model = load_model("w2v.h5")
w2vmodel = gensim.models.Word2Vec.load("w2v.bin")

# Now you can use the loaded_model for predictions
new_sentences = ['melihat layar komputer dengan dekat dapat merusak penglihatan namun tidak merusak mata secara permanen', 'diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan', 'diabetes dapat mempengaruhi kesehatan mata kondisi ini dapat menyebabkan retinopati diabetik yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan']
new_vectors = get_avg_word_vectors(new_sentences, w2vmodel)
predictions = loaded_model.predict(new_vectors)

for sentence, prediction in zip(new_sentences, predictions):
    sentiment = "Positive" if prediction[0] == 1 else "Negative"
    print(sentence, prediction)
    # print(f"Sentence: {sentence} - Sentiment: {sentiment}")

1/1 [==============================] - 0s 104ms/step
membaca dalam cahaya redup akan merusak mata ini hanya menyebabkan ketegangan mata sementara tetapi tidak merusak mata secara permanen [0.49832264]
melihat layar komputer dengan dekat dapat merusak penglihatan namun tidak merusak mata secara permanen [0.49784335]
diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan [0.50058997]
diabetes dapat mempengaruhi kesehatan mata kondisi ini dapat menyebabkan retinopati diabetik yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan [0.50093895]
